In [1]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
french_stopwords = list(fr_stop)
from scipy import stats
import math
import string

In [2]:
def LoadJsonFile(filename): 
    with open(filename, 'r') as f:
        DicConfig = json.load(f)
    return DicConfig


def GlobalDicDeplier(OneDic):
    for k,v in OneDic.items():
        exec('globals()[k] = v')
    return None

In [3]:
DicConfig = LoadJsonFile(os.path.join(os.getcwd(),"config.json"))
GlobalDicDeplier(DicConfig)
sys.path.append(Root)
from fun import *

print("Load Config variables")

Load Config variables


# Import Data

In [4]:
reffam = PickleLoad(os.path.join(Root,FolderProject,"RefFam.pkl"))
refrt = PickleLoad(os.path.join(Root,FolderProject,"RefRT.pkl"))
refrt["EPOK"] = refrt.TWEETUNIXEPOCH / 86400
refrt["EPOK"] = refrt.EPOK.astype(int)
TweetsStats = refrt.groupby(["AUTHORTWEETID","EPOK"]).size().reset_index().rename(columns = {0:"w"})
TweetsToAnalyse = reffam[["AUTHORTWEETID","AUTHORTWEETCONTENT","AUTHORTWEETUNIXEPOCH"]].\
rename(columns = {"AUTHORTWEETID":"TWEETID","AUTHORTWEETCONTENT":"TWEETCONTENT"})

In [5]:
refrt.shape

(493208, 7)

# Compute tfidf score

In [202]:
corpus = Corpus(RemoveWordsPeriod,TooFrequentThreshold,TooInfrequentThreshold)

In [203]:
corpus.LoadData()

No data!


In [204]:
corpus.ProcessCorpus(TweetsToAnalyse)

In [205]:
corpus.ComputeCorpus()

1 / 3
2 / 3
3 / 3


In [206]:
corpus.SaveOnDisk()

In [207]:
tfidf = corpus.DocsRepresentation

In [208]:
tfidf.shape

(24857, 6)

# Add some informations

In [211]:
%%time
tfidf = tfidf.merge(TweetsStats,left_on="tweetid",right_on="AUTHORTWEETID")
tfidf.sort_values(by=["w","tweetid"],inplace=True,ascending=False)

CPU times: user 1.21 s, sys: 36.4 ms, total: 1.24 s
Wall time: 1.34 s


### Calcul du nuage de mots

In [241]:
corpus.COMPTEUR['salamé']

9

In [246]:
tfidf[tfidf.tweetid == 1223007786315014151]

,D,Rank,f,idf,tweetid,word,AUTHORTWEETID,EPOK,w,power
27543,8793,1,7,7,1223007786315014144,mohamed,1223007786315014144,18292,259,1848
27545,8793,2,42,5,1223007786315014144,tête,1223007786315014144,18292,259,1384
27547,8793,3,44,5,1223007786315014144,appelle,1223007786315014144,18292,259,1372
27549,8793,4,60,5,1223007786315014144,mieux,1223007786315014144,18292,259,1292
27551,8793,5,106,4,1223007786315014144,musulmans,1223007786315014144,18292,259,1144
27553,8793,6,108,4,1223007786315014144,êtes,1223007786315014144,18292,259,1139
27542,8793,1,7,7,1223007786315014144,mohamed,1223007786315014144,18291,66,471
27544,8793,2,42,5,1223007786315014144,tête,1223007786315014144,18291,66,353
27546,8793,3,44,5,1223007786315014144,appelle,1223007786315014144,18291,66,350
27548,8793,4,60,5,1223007786315014144,mieux,1223007786315014144,18291,66,329


In [260]:
pd.to_datetime(refrt.TWEETUNIXEPOCH[refrt.AUTHORTWEETID==1223210808169070592].min(),unit="s")

Timestamp('2020-01-31 12:48:31')

In [261]:
pd.to_datetime(refrt.TWEETUNIXEPOCH[refrt.AUTHORTWEETID==1223210808169070592].max(),unit="s")

Timestamp('2020-01-31 20:55:06')

In [259]:
tfidf[tfidf.word=="mila"]

,D,Rank,f,idf,tweetid,word,AUTHORTWEETID,EPOK,w,power
20428,8793,4,136,4,1223510356439568384,mila,1223510356439568384,18293,432,1801
25154,8793,4,136,4,1223210808169070592,mila,1223210808169070592,18292,424,1768
20337,8793,4,136,4,1223517174683328256,mila,1223517174683328256,18293,379,1580
20155,8793,4,136,4,1223532135539204096,mila,1223532135539204096,18293,312,1301
12184,8793,3,136,4,1223990092861202432,mila,1223990092861202432,18294,292,1217
...,...,...,...,...,...,...,...,...,...,...
16211,8793,4,136,4,1223699982777298944,mila,1223699982777298944,18294,1,4
17447,8793,3,136,4,1223657376768843520,mila,1223657376768843520,18294,1,4
17820,8793,4,136,4,1223644846486949888,mila,1223644846486949888,18294,1,4
18591,8793,7,136,4,1223622032249638656,mila,1223622032249638656,18294,1,4


In [243]:
Tokenizer(reffam.AUTHORTWEETCONTENT[reffam.AUTHORTWEETID==1223007786315014144].iloc[0])

['tête',
 'appelle',
 'mohamed',
 'crache',
 'musulmans',
 'mieux',
 'êtes',
 'https',
 'mzjphegtsq']

In [245]:
reffam.AUTHORTWEETCONTENT[reffam.AUTHORTWEETID==1223007786315014144].iloc[0]

'"Dans la tête des gens comme il s\'appelle Mohamed s\'il crache sur les musulmans, c\'est encore mieux. Vous êtes un p… https://t.co/mzJPHegtSQ'

In [212]:
tfidf["power"] = tfidf.w * tfidf.idf
tfidf.head()

,D,Rank,f,idf,tweetid,word,AUTHORTWEETID,EPOK,w,power
13391,8793,1,706,3,1223938187355140096,wallah,1223938187355140096,18294,6399,16139
19810,8793,1,15,6,1223557159285337856,lycéens,1223557159285337856,18293,3194,20357
19811,8793,2,19,6,1223557159285337856,garde,1223557159285337856,18293,3194,19602
19812,8793,3,23,6,1223557159285337856,vue,1223557159285337856,18293,3194,18992
19813,8793,4,25,6,1223557159285337856,place,1223557159285337856,18293,3194,18726


In [213]:
wordcloud = tfidf.groupby(["word","EPOK"])["power"].sum().reset_index()

In [214]:
wordcloud.sort_values(by=["EPOK","power"],ascending=False,inplace=True)

In [216]:
wordcloud["rank"] = wordcloud.groupby(["EPOK"]).cumcount()+1
wordcloud.reset_index(drop=True,inplace=True)

In [218]:
for e in wordcloud.EPOK.unique():
    print(wordcloud[wordcloud.EPOK==e].head(20))
    print("")

          word   EPOK  power  rank
0        allah  18294  26703     1
1       wallah  18294  25874     2
2      londres  18294  25809     3
3      attaque  18294  19625     4
4      couteau  18294  17887     5
5    personnes  18294  16392     6
6   terroriste  18294  15936     7
7       police  18294  15643     8
8        islam  18294  13209     9
9          ans  18294  12693    10
10     haddach  18294  12612    11
11      rachid  18294  12472    12
12      france  18294  11827    13
13    vraiment  18294  11018    14
14      abattu  18294  11016    15
15      jamais  18294  10516    16
16       homme  18294   9881    17
17      femmes  18294   9797    18
18    attentat  18294   9742    19
19    français  18294   9632    20

           word   EPOK  power  rank
507  traduction  18293  29196     1
508     lycéens  18293  25979     2
509       garde  18293  24690     3
510         vue  18293  23161     4
511       place  18293  20878     5
512       vient  18293  19006     6
513       is

## Random tweet analysis

In [169]:
mytweetid = np.random.choice(a = tfidf.tweetid.unique())
mytweetid = 1224088992150409216
content,tokens,details = ExploreTweetTreatment(mytweetid,reffam,tfidf)

KeyError: "['w'] not in index"

In [114]:
print(content)

Affaire #Mila : la Parquet fait machine arrière, #Belloubet se déjuge et les islamo-gauchistes font grise mine.
Con… https://t.co/Un4483y1j4


In [115]:
print(tokens)

['affaire', 'mila', 'parquet', 'machine', 'arrière', 'belloubet', 'déjuge', 'islamo', 'gauchistes', 'grise', 'mine', 'con', 'https', 'un4483y1j4']


In [116]:
details

[{'word': 'parquet',
  'idf': 7.477523438381578,
  'Rank': 1,
  'f': 7,
  'w': 116,
  'WordPercentile': 0.059632540356839424},
 {'word': 'belloubet',
  'idf': 7.226209010100671,
  'Rank': 2,
  'f': 9,
  'w': 116,
  'WordPercentile': 0.07735060889266497},
 {'word': 'islamo',
  'idf': 5.989446382951745,
  'Rank': 3,
  'f': 31,
  'w': 116,
  'WordPercentile': 0.20897054658736902},
 {'word': 'affaire',
  'idf': 5.079628165583207,
  'Rank': 4,
  'f': 77,
  'w': 116,
  'WordPercentile': 0.4540321438685925},
 {'word': 'mila',
  'idf': 4.186991624606942,
  'Rank': 5,
  'f': 188,
  'w': 116,
  'WordPercentile': 0.728529453412631}]

In [168]:
def ExploreTweetTreatment(mytweetid,reffam,tfidf):
    content = reffam.AUTHORTWEETCONTENT[reffam.AUTHORTWEETID==mytweetid].iloc[0]
    tokens = Tokenizer(content)
    details = tfidf[tfidf.tweetid==mytweetid][["word","idf","Rank","f","w"]].\
    to_dict(orient="records")
    return content,tokens,details

In [78]:
class Corpus():
    
    # Initialisation de l'objet
    def __init__(self,RemoveWordsPeriod,TooFrequentThreshold,TooInfrequentThreshold):
        
        self.WORDS2TWEETS = pd.DataFrame()
        self.DOCSTOUCHED = pd.DataFrame()
        self.COMPTEUR = {}
        self.BATCH_WORDS2TWEETS = pd.DataFrame()
        self.BATCH_DOCSTOUCHED = pd.DataFrame()
        self.BATCH_COMPTEUR = {}
        self.BATCH_V = []
        self.DocsRepresentation = pd.DataFrame()
        self.RemoveWordsPeriod = RemoveWordsPeriod
        self.TooFrequentThreshold = TooFrequentThreshold
        self.TooInfrequentThreshold = TooInfrequentThreshold
        
        self.LastDate = 0
     
    
    def LoadData(self):
        
        self.WORDS2TWEETS = LoadPickleOrInit(os.path.join(Root,FolderProject,"tfidf_words2tweets.pkl"))
        self.DOCSTOUCHED = LoadPickleOrInit(os.path.join(Root,FolderProject,"tfidf_docstouched.pkl"))
        self.COMPTEUR = LoadPickleOrInit(os.path.join(Root,FolderProject,"tfidf_compteur.pkl"),typeobj="dic")
        self.DocsRepresentation = LoadPickleOrInit(os.path.join(Root,FolderProject,"tfidf_DocsRepresentation.pkl"))
        self.LastDate = LoadPickleOrInit(os.path.join(Root,FolderProject,"tfidf_lastdate.pkl"),typeobj="0")
        
        if len(self.DOCSTOUCHED)==0:
            print("No data!")
            
        return None
    
    
    def SaveOnDisk(self):
        
        PickleDump(os.path.join(Root,FolderProject,"tfidf_compteur.pkl"),self.COMPTEUR)
        PickleDump(os.path.join(Root,FolderProject,"tfidf_docstouched.pkl"),self.DOCSTOUCHED)
        PickleDump(os.path.join(Root,FolderProject,"tfidf_words2tweets.pkl"),self.WORDS2TWEETS)
        PickleDump(os.path.join(Root,FolderProject,"tfidf_DocsRepresentation.pkl"),self.DocsRepresentation)
        PickleDump(os.path.join(Root,FolderProject,"tfidf_lastdate.pkl"),self.LastDate)
        
        return None
    
    
    # EVALUATION de la variable globale |D|
    # i.e. le nombre de documents, i.e. les tweets
    def EvaluateD(self):
        if len(self.WORDS2TWEETS)>0:
            res = len(np.unique(self.WORDS2TWEETS.tweetid))
        else:
            res = 0
        return res
    
    # CREATION de la liste de batch
    def ProcessCorpus(self,TweetsDataFrame):
        
        tweetsdf = TweetsDataFrame.copy()
        
        if len(self.WORDS2TWEETS)>0:
            a = self.WORDS2TWEETS.tweetid.unique()
            b = tweetsdf.TWEETID
            fil = ~b.isin(a)
            tweetsdf = tweetsdf[fil]
            
        tweetsdf = tweetsdf.copy()[tweetsdf.AUTHORTWEETUNIXEPOCH>self.LastDate]
        ListOfTweetsDF = SplitTweetsToAnalyse(tweetsdf,self.RemoveWordsPeriod)
        self.WaitingCorpusList = ListOfTweetsDF
        
        if len(ListOfTweetsDF)>0:
            self.LastDate = self.WaitingCorpusList[-1].AUTHORTWEETUNIXEPOCH.max()
        
        return None
        
        
    # TRAITEMENT DE LA LISTE DE BATCH
    def ComputeCorpus(self):
        for iddf,df in enumerate(self.WaitingCorpusList):
            print(str(iddf+1),"/",str(len(self.WaitingCorpusList)))
            self.AddCorpus(df)
            self.BuildDocsRepresentation()
        return None
            
            
    # TRAITEMENT D'UN BATCH
    def AddCorpus(self,tweetsdf):     
        
        if len(tweetsdf)>0:
            
            # Build les batch df
            tweetsdf.TWEETCONTENT = tweetsdf.TWEETCONTENT.map(lambda a : Tokenizer(a))
            Words2TweetsDF = DeplyrDF(tweetsdf)
            CompteurDic = BooleanCorpusCompteur(tweetsdf.TWEETCONTENT.tolist())
            V = list(CompteurDic.keys())
            DocsTouched = BuildDocumentsTouched(V,tweetsdf.TWEETCONTENT.tolist())
            DocsTouched = pd.DataFrame(data=DocsTouched,index=[0]).T.reset_index().rename(columns = {"index":"word",0:"f"})
            
            # Remove les mots trop peu fréquents du batch
            TooFrequentRaw = int(self.TooFrequentThreshold * len(Words2TweetsDF.tweetid.unique()))
            TooInfrequentRaw = int(self.TooInfrequentThreshold * len(Words2TweetsDF.tweetid.unique()))
            DocsTouched = DocsTouched.copy()[(DocsTouched.f>TooInfrequentRaw) & (DocsTouched.f<TooFrequentRaw)]
            Words2TweetsDF = Words2TweetsDF.merge(DocsTouched,on="word")
            Words2TweetsDF = Words2TweetsDF[["word","tweetid"]]
            V = DocsTouched.word.tolist()
            
            # Assignation des batch df
            self.BATCH_WORDS2TWEETS = Words2TweetsDF
            self.BATCH_DOCSTOUCHED = DocsTouched
            self.BATCH_COMPTEUR = CompteurDic
            self.BATCH_V = V

            
                                                                 
                                                                 
            #  * * * UPDATING PART * * * 
            #  * * * UPDATING PART * * *
            #  * * * UPDATING PART * * *
            
            
            
            
            # maj du dic
            self.COMPTEUR = UpdateCompteurDic(self.COMPTEUR,self.BATCH_COMPTEUR)
            
            # maj du words2tweets
            self.WORDS2TWEETS = pd.concat((self.WORDS2TWEETS,self.BATCH_WORDS2TWEETS),axis=0)
            
            # maj du docstouched
            if len(self.DOCSTOUCHED)>0:
                fil = self.DOCSTOUCHED.word.isin(pd.Series(V))
                nepastoucher = self.DOCSTOUCHED[~fil]
                amodifier = self.DOCSTOUCHED[fil]
                amodifier = pd.concat((amodifier,self.BATCH_DOCSTOUCHED),axis = 0)
                amodifier = amodifier.groupby("word")["f"].sum().reset_index()
                self.DOCSTOUCHED = pd.concat((amodifier,nepastoucher),axis = 0)
            else:
                self.DOCSTOUCHED = self.BATCH_DOCSTOUCHED
                

            
        # si tweetsdf est vide alors les batch sont vides
        else:
            self.BATCH_WORDS2TWEETS = pd.DataFrame()
            self.BATCH_DOCSTOUCHED = pd.DataFrame()
            self.BATCH_COMPTEUR = {}
            self.BATCH_V = []
        
        return None
    
    

    

    
    
    
    def BuildDocsRepresentation(self):
        
        # Si y'a rien dans le batch alors on s'arrête là
        if len(self.BATCH_V)==0:
            return None
        
        # Si y'a déjà des idf scores de calculé on doit la mettre à jour
        # Le keepdf, contient les mots non présents dans le batch
        if len(self.DocsRepresentation)>0:
            fil = self.DocsRepresentation.word.isin(pd.Series(self.BATCH_V))
            DocsRepresentationKeep = self.DocsRepresentation.copy()[~fil]
            DocsRepresentationKeep["D"] = self.EvaluateD()
        else:
            DocsRepresentationKeep = pd.DataFrame()
        
        # df intermediaires
        fil = self.DOCSTOUCHED.word.isin(pd.Series(self.BATCH_V))
        temptouched = self.DOCSTOUCHED.copy()[fil]
        fil = self.WORDS2TWEETS.word.isin(pd.Series(self.BATCH_V))
        tempwords2 = self.WORDS2TWEETS[fil]        
        toadd = temptouched.merge(tempwords2,on="word")
        toadd["D"] = self.EvaluateD()   
        
        # concaténation des scores des anciens mots et des nouveaux mots
        solution = pd.concat((toadd,DocsRepresentationKeep),axis = 0, sort = True)    
        solution["idf"] = (solution.D / solution.f).map(lambda a : math.log(a))
        solution = solution
        solution.reset_index(drop = True,inplace = True)
        solution.sort_values(by = ["tweetid","idf"],ascending=False,inplace=True)
        solution.drop_duplicates(inplace = True)
        solution['Rank'] = solution.groupby(["tweetid"]).cumcount()+1
        self.DocsRepresentation = solution
        
        return None
        
        
        
        
        
        
        

In [79]:
def Tokenizer(randomstring,french_stopwords=french_stopwords):
    translator = str.maketrans(string.punctuation,' '*32)
    randomstring = randomstring.translate(translator)
    randomstring = randomstring.replace("’"," ")
    randomstring = randomstring.replace("`"," ")
    randomstring = randomstring.replace("'"," ")
    randomstring = randomstring.replace("“"," ")
    randomstring = randomstring.replace("”"," ")
    randomstring = randomstring.replace("…"," ")
    randomstring = randomstring.lower()
    randomstring = " ".join(randomstring.split())
    words = randomstring.split(" ")
    words = [item for item in words if item not in french_stopwords]
    words = [item for item in words if len(item)>2]
    return words

In [80]:
def SentenceCompteur(compteur,words,weight=1):
    for w in words:
        compteur[w] = compteur.get(w,0) + weight
    return compteur

def BooleanCorpusCompteur(Corpus):
    compteur = {}
    for doc in Corpus:
        compteur = SentenceCompteur(compteur,doc)
    return compteur

In [81]:
def BuildDocumentsTouched(V,CleanedCorpus):
    
    DocumentsTouched = {}
    for w in V:
        for d in CleanedCorpus:
            if w in d:
                DocumentsTouched[w] = DocumentsTouched.get(w,0) + 1        
    
    return DocumentsTouched

In [82]:
def UpdateCompteurDic(OriginalDic,AddDic):
    for k,v in AddDic.items():
        OriginalDic[k] = OriginalDic.get(k,0) + v
    return OriginalDic

In [83]:
def DeplyrDF(TweetsDataFrame):

    L = []
    for i,row in TweetsDataFrame.iterrows():
        tweetid = row["TWEETID"]
        tweetcontent = row["TWEETCONTENT"]
        tweetcontent = pd.Series(tweetcontent)
        tempdf = tweetcontent.to_frame(name = "word")
        tempdf["tweetid"] = tweetid
        L.append(tempdf)

    res = pd.concat(L,axis=0)
    res.reset_index(drop=True,inplace=True)

    return res

In [84]:
def SplitTweetsToAnalyse(TweetsToAnalyse,RemoveWordsPeriod):
    TweetsToAnalyse["TimeElapsed"] = (TweetsToAnalyse.AUTHORTWEETUNIXEPOCH.max() - TweetsToAnalyse.AUTHORTWEETUNIXEPOCH) / RemoveWordsPeriod
    TweetsToAnalyse["TimeElapsed"] = TweetsToAnalyse["TimeElapsed"].astype(int)
    idgroup = TweetsToAnalyse.TimeElapsed.unique()
    idgroup.sort()
    L = []
    for idg in idgroup[:-1]:
        tempdf = TweetsToAnalyse.copy()[TweetsToAnalyse.TimeElapsed==idg]
        L.append(tempdf)
    return L